# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL. Make sure to provide insights and alerts about significant market opportunities or threats."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and analyze it using statistical modeling and machine learning to identify trends and predict market movements. I 

Final Answer: Apple Inc. (AAPL) is currently trading at $227.63, with a decrease of 2.40%. The market capitalization is $3.419 trillion, and the company offers a range of products including iPhones, Macs, iPads, wearables, and accessories. Apple Inc. also provides various subscription-based services like Apple Music, Apple TV+, and Apple Pay. The company's profitability metrics show a profit margin of 23.97%, return on assets of 21.46%, and return on equity of 157.41%. Additionally, the stock has shown a YTD return of -9.10% and a 1-year return of 20.18%. It is important to monitor these metrics along with market trends and news to predict market movements accurately.

> Finished chain.
 

Apple Inc. (AAPL) is currently trading at $227.63, with a decrease of 2.40%. The market capitalization is $3.419 trillion, and the company offers a range of products including iPhones, Macs, iPads, wearables, and accessories. Apple Inc. also provides various subscription-based services like Apple Mus

I need to gather more information about AAPL's market performance and trends to provide valuable insights for trading strategy development.

Action: Read website content
Action Input: {"website_url": "https://www.marketwatch.com/investing/stock/aapl"} 

marketwatch.comPlease enable JS and disable any ad blocker

Final Answer: marketwatch.comPlease enable JS and disable any ad blocker

> Finished chain.
 

marketwatch.comPlease enable JS and disable any ad blocker

The Data Analyst has started analyzing AAPL's market performance and trends for trading strategy development.

Thought: 
I also need to delegate tasks to the Trading Strategy Developer to create potential trading strategies for AAPL that align with the user's risk tolerance and trading preferences.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop potential trading strategies for AAPL based on user-defined risk tolerance and trading preferences", "context": "AAPL is 

I need to gather more insights from my team members to create detailed execution plans for the trading strategies for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "question": "Can you provide detailed execution plans for the Long-Term Investment, Options Trading, and Dividend Investing strategies for AAPL?", "context": "We need to determine how and when to execute trades for AAPL based on the approved trading strategies."}

> Entering new CrewAgentExecutor chain...
I should gather information on Long-Term Investment, Options Trading, and Dividend Investing strategies for AAPL to provide detailed execution plans.

Action: Read website content
Action Input: {"website_url": "https://www.investopedia.com/articles/investing/102214/investing-apple-inc-aapl-stock.asp"} 

Page Not Found
Skip to content
 Investing
 Stocks
 Cryptocurrency
 Bonds
 ETFs
 Options and Derivatives
 Commodities
 Trading
 Automated Investing
 Brokers
 Fundamental Ana

Thought: I have gathered valuable insights on the proposed trading strategies and execution plans for AAPL through the search results.

Final Answer: After conducting a search on the internet, I found various trading strategies and insights related to AAPL, including scalping, swing trading, options strategies, and technical analysis. These strategies involve executing multiple trades within a short time frame, maximizing profits with swing trading, and utilizing different approaches to market analysis. It is crucial to consider the potential risks associated with each strategy, such as market volatility, price differentials, and earnings outcomes, to develop effective mitigation strategies and align trading activities with the firm's risk tolerance.

> Finished chain.
 

After conducting a search on the internet, I found various trading strategies and insights related to AAPL, including scalping, swing trading, options strategies, and technical analysis. These strategies involve execu

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL:

After a thorough evaluation of the proposed trading strategies and execution plans for AAPL, the following potential risks have been identified along with suggested mitigation strategies:

1. Market Volatility: The volatile nature of the stock market can lead to significant fluctuations in AAPL's stock price. Mitigation Strategy: Implement stop-loss orders and utilize hedging strategies to minimize losses during periods of high volatility.

2. Price Differentials: Price differentials between AAPL's stock price on different trading platforms can create arbitrage opportunities but also pose risks of delayed execution and potential losses. Mitigation Strategy: Monitor price differentials closely and use algorithmic trading tools to capitalize on arbitrage opportunities efficiently.

3. Earnings Outcomes: AAPL's stock price is highly influenced by its quarterly earnings reports, which can lead to sharp movements in the stock price. Mitigation Strategy: Diversify the trading portfolio to reduce reliance on AAPL's performance alone and consider the impact of earnings outcomes on trading strategies.

4. Scalping Strategy: The scalping strategy involves making quick trades to capitalize on small price movements, but it also carries the risk of high transaction costs and slippage. Mitigation Strategy: Use automated trading tools to execute scalping trades efficiently and minimize transaction costs.

5. Swing Trading Strategy: Swing trading aims to capture short to medium-term gains in AAPL's stock price, but it requires careful analysis of market trends and risk management. Mitigation Strategy: Set clear entry and exit points for swing trades and use technical indicators to identify potential reversals in the stock price.

6. Options Strategies: Options trading involves complex financial instruments that can amplify gains but also lead to significant losses if not managed properly. Mitigation Strategy: Educate traders on the risks and rewards of options trading, implement strict risk management protocols, and consider using options strategies as part of a diversified trading portfolio.

In conclusion, it is essential to consider the potential risks associated with each trading strategy and execution plan for AAPL to develop effective mitigation strategies and align trading activities with the firm's risk tolerance. By implementing the suggested mitigation strategies, the team can navigate the risks effectively and optimize trading performance for AAPL.